In [ ]:
# import sys
# sys.path.append('../data')
# print each command: 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [11]:
# General DS libraries we are going to need
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timedelta

# Importing our base model
# [REDACTED ML MODEL USED]

# Imports for working with our large dataset
from sklearn.utils.random import sample_without_replacement
from sklearn.model_selection import train_test_split

# We need those for data manipulation and getting our features ready for the model
from sklearn.preprocessing import OneHotEncoder, Normalizer, Binarizer
from sklearn.compose import make_column_transformer

# These can be used to measure our model's performance
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix

# Ignore DataFrame assignment warnings
pd.options.mode.chained_assignment = None

pd.options.display.float_format = '{:,.2f}'.format

In [3]:
n_neighbors = 7 
group_name = "first" 
samples_num = 100000 
n_jobs = 1 
path_prefix = "./data"
nrows=2e6

In [4]:
%%time
train_accounts = pd.read_csv('{}/train_accounts.csv'.format(path_prefix), low_memory=False, parse_dates=['created_at'], infer_datetime_format=True, nrows=nrows)
train_users = pd.read_csv('{}/train_users.csv'.format(path_prefix), low_memory=False, parse_dates=['created_at'], infer_datetime_format=True, nrows=nrows)
train_events = pd.read_csv('{}/train_events.csv'.format(path_prefix), low_memory=False, parse_dates=['date'], infer_datetime_format=True, date_parser=(lambda x: pd.to_datetime(x, errors='coerce')), nrows=nrows)
train_subscriptions = pd.read_csv('{}/train_subscriptions.csv'.format(path_prefix), low_memory=False, parse_dates=['event_happened_at'], infer_datetime_format=True, nrows=nrows)

Wall time: 21.4 s


In [5]:
%%time
test_accounts = pd.read_csv('{}/test_accounts.csv'.format(path_prefix), low_memory=False, parse_dates=['created_at'], infer_datetime_format=True)
test_users = pd.read_csv('{}/test_users.csv'.format(path_prefix), low_memory=False, parse_dates=['created_at'], infer_datetime_format=True)
test_events = pd.read_csv('{}/test_events.csv'.format(path_prefix), low_memory=False, parse_dates=['date'], infer_datetime_format=True, date_parser=(lambda x: pd.to_datetime(x, errors='coerce')))
test_subscriptions = pd.read_csv('{}/test_subscriptions.csv'.format(path_prefix), low_memory=False, parse_dates=['event_happened_at'], infer_datetime_format=True)

Wall time: 3.38 s


In [7]:
accounts = pd.concat([train_accounts, test_accounts],sort=False)
users = pd.concat([train_users, test_users],sort=False)
events = pd.concat([train_events, test_events],sort=False)
subscriptions = pd.concat([train_subscriptions, test_subscriptions],sort=False)

In [12]:
accounts.nunique()
accounts.describe()

account_id                 1433659
account_name                533308
created_at                 1357561
plan_id                        113
trial_start                1360617
churn_date                    3007
churn_reason                    18
time_diff                       41
region                        3601
country                        236
subscription_started_at      33797
paying                           2
has_logo                         1
device                           3
os                               7
browser                         12
collection_21_days            1490
company_size                  4254
payment_currency                 8
max_team_size                   14
min_team_size                   15
industry                        32
utm_cluster_id                  27
mrr                            385
user_goal                        3
user_description                 5
team_size                       19
lead_score                       2
dtype: int64

,account_id,plan_id,churn_reason,time_diff,paying,has_logo,collection_21_days,company_size,max_team_size,min_team_size,mrr,lead_score
count,"1,433,661.00","30,089.00","25,256.00","1,191,655.00","1,433,661.00","1,433,661.00","1,433,661.00","298,488.00","1,278,324.00","1,278,324.00","48,000.00","1,361,978.00"
mean,"716,829.94",441.73,14.75,-1.44,0.02,1.00,7.70,"10,992.29",17.60,10.93,38.80,0.02
std,"413,861.68",412.17,3.27,4.72,0.14,0.00,102.09,"59,591.59",68.53,53.28,68.70,0.16
min,1.00,9.00,1.00,-12.00,0.00,1.00,"-1,440.00",1.00,0.00,0.00,0.00,0.00
25%,"358,416.00",208.00,13.00,-5.00,0.00,1.00,0.00,30.00,1.00,1.00,0.00,0.00
50%,"716,830.00",232.00,15.00,-3.00,0.00,1.00,0.00,210.00,5.00,2.00,25.00,0.00
75%,"1,075,244.00",816.00,18.00,2.00,0.00,1.00,0.00,"2,750.00",10.00,6.00,48.00,0.00
max,"1,433,659.00","1,678.00",18.00,14.00,1.00,1.00,"20,808.00","2,300,000.00",500.00,500.00,"3,468.00",1.00


In [68]:
# users[accounts[account_id]==1000]
# accounts.head()
users.loc[:,"account_id"]

0                1.00
1                2.00
2                2.00
3                3.00
4                4.00
             ...     
111140   1,433,590.00
111141   1,433,591.00
111142   1,433,599.00
111143   1,433,645.00
111144   1,433,654.00
Name: account_id, Length: 2111145, dtype: float64